In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [15]:
img = cv2.imread("lenna.png",0)

In [108]:
def floyd_steinberg(original_img,thresh=128,zig_zag=False):
    dithering = np.zeros((original_img.shape[0]+1,original_img.shape[1]+1), np.uint8)
    img = original_img.copy()
    img.resize(original_img.shape[0]+1,original_img.shape[1]+1)
    
    for y in range(img.shape[0]-1):
        for i in range(img.shape[1]-1):
            if(zig_zag == True and i%2 == 0):
                x = img.shape[1] - i
            else:
                x = i
            err = 0
            if(img[y][x] > thresh):
                dithering[y][x] = 255
                err = img[y][x] - 255
            else:
                dithering[y][x] = 0
                err = img[y][x] - 0

                img[y][x+1] = img[y][x+1] + err*(7/16)
                img[y+1][x-1] = img[y+1][x-1] + err*(3/16)
                img[y+1][x] = img[y+1][x] + err*(5/16)
                img[y+1][x+1] = img[y+1][x+1] + err*(1/16)
    return dithering

In [109]:
a0 = floyd_steinberg(img)
img = cv2.imread("lenna.png",0)
a1 = floyd_steinberg(img,zig_zag=True)

[[169 169 168 ... 162 138 169]
 [169 168 167 ... 138 169 169]
 [168 167 169 ... 169 169 168]
 ...
 [118 122  53 ... 115 117 118]
 [122   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
[[169 169 168 ... 162 138 169]
 [169 168 167 ... 138 169 169]
 [168 167 169 ... 169 169 168]
 ...
 [118 122  53 ... 115 117 118]
 [122   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]


IndexError: index 513 is out of bounds for axis 0 with size 513

In [ ]:
cv2.imwrite("floyd.png",a0)
cv2.imwrite("floyd_flip.png",a1)

In [39]:
d = {0:[[0,0,0],
        [0,0,0],
        [0,0,0]],
     
     1:[[0,255,0],
        [0,0,0],
        [0,0,0]],
     
     2:[[0,255,0],
        [0,0,0],
        [0,0,255]],
     
     3:[[255,255,0],
        [0,0,0],
        [0,0,255]],
     
     4:[[255,255,0],
        [0,0,0],
        [255,0,255]],
     
     5:[[255,255,255],
        [0,0,0],
        [255,0,255]],
     
     6:[[255,255,255],
        [0,0,255],
        [255,0,255]],
     
     7:[[255,255,255],
        [0,0,255],
        [255,255,255]],
     
     8:[[255,255,255],
        [255,0,255],
        [255,255,255]],
     
     9:[[255,255,255],
        [255,255,255],
        [255,255,255]],
     }

In [40]:
def halftoning(img):
    new_img = np.zeros([img.shape[0]*3,img.shape[1]*3])
    m = 1
    n = 1
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            cols = np.array([[m-1, m, m+1],
                             [m-1, m, m+1],
                             [m-1, m, m+1]],
                            dtype=np.intp)
            rows = np.array([[n-1, n-1, n-1],
                             [n, n, n],
                             [n+1, n+1, n+1]],
                             dtype=np.intp)
            
            new_img[cols,rows] = d[round(np.interp(img[i][j],[0,255],[0,9]))]
            n = (n+3)%(new_img.shape[1])
        m = (m+3)%(new_img.shape[0])
    return new_img

In [41]:
img = cv2.imread("lenna.png",0)
a = halftoning(img)

In [48]:
cv2.imwrite("dithering.png",a)

True

In [114]:
a = np.ones([3])
print(a.resize([4]))

None
